# CSC421 Fall 2021 Assignment 2 
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 7 - Logical Agents** and **Chapter 8 - First-Order Logic** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the code provided in logic.py and logic.ipynb for completing the assignment questions. The first 5 questions do NOT rely on the provided aima code so you can complete it just using basic Python. The next 5 questions require the use of the aima-code repository. 

You are welcome and actually it can be educational to look at the code at the aima-code repository as well as other code resources you can find on the web. However, make sure you understand any code that you incoporate. 

The assignment structure is as follows - each item is worth 1 point: 

1. Propositional Logic (Basic) - simple infix evaluator for 0, 1 and logical operators 
2. Propositional Logic (Basic) - adding variables and bindings to evaluator 
3. Propositional Logic (Expected) - recursive infix evaluator for propositional logic 
4. Propositional Logic (Expected) - usage of evaluator to evaluate some example logic expressions
5. Propositional Logic (Advanced) - model checking for the prefix evaluator 
6. Proposigtional Logic(Basic) - simple KB using aima code with model checking and theorem proving 
7. First-Order Logic (Basic) - kinship domain using FolKB for the Simpsons 
8. First-Order Logic (Expected) - kinship domain using Prolog for the Simpsons
9. First-Order Logic (Expected) - LegoWorld 
10.First-Order Logic (Advanced) - NLP for LegoWorld or webscraping a KB for music  

The grading will be done in 0.5 increments. 1 point for correct answer, 0.5 points for partial or incorrect 
but reasonable answer and 0.0 for no answer or completely wrong answer. 

```
Birds can fly, unless they are penguins and ostriches, or if they happen 
to be dead, or have broken wings, or are confined to cages, or have their 
feet stuck in cement, or have undergone experiences so dreadful as to render 
them psychologically incapable of flight 

Marvin Minsky 
```

In [1]:
!pip install numpy networkx ipythonblocks sortedcontainers

  Using cached numpy-1.21.3-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached ipythonblocks-1.9.0-py2.py3-none-any.whl (13 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


numpy networkx ipythonblocks sortedcontainers# Introduction - Parsing and evaluating prefix logic expressions  

In this assignment your task is to incrementally create a parser and evaluator for prefix logic expressions as well as implement simple model checking. 


# Question 1 (Basic)  - 1 point

Your first task will be to write a simple evaluator of prefix logic expressions with constants. In prefix notation the operator precedes the operands and no operands are required. For example 5+3 in prefix notation is written + 5 3 or 5 * 2 + 3 would be written + * 5 2 3 or + * 5 2 * 3 4 is equivalent to (5 * 2) + (3 * 4). 

As a first step we will consider very simple expressions with one operator and two constant operands. We will use 0 for false and 1 for true. The following logical connectives should be implemented (see Figure 7.8 in your book) (notice that for now there is no negation symbol ~): 

1. &    (and), 
3. |    (or), 
4. =>   (implication) 
5. <=>  (biconditional) 

Example expressions: 
```
& 1 0  
=> 0 1 
<=> 1 1 
```

Your function should take as input a string with the prefix expression and return the result of evaluating the expression (basically a 1 for true and 0 for false). You can split a string to a list using .split[' ']. For this part of the assignment you only evaluate expressions with two constant operands i.e no nested/recursive expressions. 

In [1]:
a = '& 1 0'
print(a.split(' '))

['&', '1', '0']


In [86]:
# YOUR CODE GOES HERE 

def simple_logic_eval(expression):
    expr_l = expression.split(' ')
    operator = expr_l[0]
    operand1 = int(expr_l[1])
    operand2 = int(expr_l[2])
    
    if operator == '&':
        return operand1 and operand2
    if operator == '|':
        return operand1 or operand2
    if operator == '=>':
        return int(not operand1) or operand2
    if operator == '<=>':
        return (int(not operand2) or operand1) and (int(not operand1) or operand2)
    else:
        print('Expression could not be evaluated.')
    

result = simple_logic_eval('& 0 1')
result9 = simple_logic_eval('& 1 1')
print('Results for &: {} when 0 1; {} when 1 1'.format(result, result9))

result2 = simple_logic_eval('| 0 0')
result8 = simple_logic_eval('| 0 1')
print('Results for |: {} when 0 0; {} when 0 1'.format(result2, result8))

result3 = simple_logic_eval('=> 1 1')
result10 = simple_logic_eval('=> 0 1')
result11 = simple_logic_eval('=> 1 0')
result12 = simple_logic_eval('=> 0 0')
print('Results for =>: {} when 1 1; {} when 1 0; {} when 1 0; {} when 0 0'.format(result3, result10, result11, result12))

result4 = simple_logic_eval('<=> 0 1')
result5 = simple_logic_eval('<=> 1 0')
result6 = simple_logic_eval('<=> 1 1')
result7 = simple_logic_eval('<=> 0 0')
print('Results for <=>: {} when 0 1; {} when 1 0; {} when 1 1; {} when 0 0.'.format(result4, result5, result6, result7))


Results for &: 0 when 0 1; 1 when 1 1
Results for |: 0 when 0 0; 1 when 0 1
Results for =>: 1 when 1 1; 1 when 1 0; 0 when 1 0; 1 when 0 0
Results for <=>: 0 when 0 1; 0 when 1 0; 1 when 1 1; 1 when 0 0.


# Question 2 (Basic) - 1 point

Your next task is to implement variables and bindings for your propositional logic evaluator. In this version in addition to constants (0 and 1) you also can have variables which are strings with associated values provided in a dictionary. You still only consider two operands and one operator (no nesting). For example in the code below 
the three expressions are equivalent. Your function should take as arguments the expression to be evaluated as a string and the dictionary with the variable bindings. In addition you need to add the ~ (not) operator. To do so for each variable in the dictionary add a not version. For example if 'a' in the dictionary has a value of 1 the '~a' in the dictionary should have a value of 0. Notice that the not symbol is part of the string and NOT separated by a space. 



In [87]:
d = {'foo': 0, 'b': 1}
print(d)
expr1 = '& 0 1'
expr2 = '& foo 1'
expr3 = '& foo ~b'

{'foo': 0, 'b': 1}


In [88]:
# YOUR CODE GOES HERE 

#returns the value of the operand from the dictionary, negates as needed 
def get_dictionary_val(op, dictionary):
    if not op.isdigit():
        if op[0] == '~':
            tmp_op = op.replace('~','')
            for k in dictionary:
                if k == tmp_op: #need to return the negated value of op
                    neg_op = not dictionary[k]
                    return int(neg_op)
                elif k == op:
                    return int(dictionary[k])
                else:
                    pass
        else:
            tmp_neg = '~'+ op
            for k in dictionary:
                if k == op:
                    return int(dictionary[k])
                elif k == tmp_neg:
                    neg_val = not dictionary[k]
                    return int(neg_val)
                else:
                    pass
    else:
        return op



#assumes any variable encounted will be in the dictionary in some form. i.e. if the entered variable is foo, then either 
#foo or ~foo will be in the dicitonary.
def expression_eval(expression, dictionary):
    epr = expression.split(' ')
    #print(epr)
    operand = epr[0]
    operand1 = epr[1]
    operand2 = epr[2]
    
    if not operand1.isdigit() or not operand2.isdigit(): 
        #get values for those that are not digits and dictionary update
        if not operand1.isdigit():
            #operand1 is a variable 
            if not operand2.isdigit():
                #both operands are variables
                val1 = get_dictionary_val(operand1, dictionary)
                val2 = get_dictionary_val(operand2, dictionary)
                expression_str = operand + ' ' + str(int(val1)) + ' ' + str(int(val2))
                return simple_logic_eval(expression_str)
            else:
                #operand1 is the lone variable 
                val3 = get_dictionary_val(operand1, dictionary)
                expression_str = operand + ' ' + str(int(val3)) + ' ' + operand2
                return simple_logic_eval(expression_str)
        else:
            #operand2 is the lone variable
            val4 = get_dictionary_val(operand2, dictionary)
            expression_str = operand + ' ' + operand1 + ' ' + str(int(val4))
            return simple_logic_eval(expression_str)          
    else:
        result = simple_logic_eval(expression)
        return result
            
    
    
#result = expression_eval(expr3, d)
print('get_dictionary_val() tests:')
result = get_dictionary_val('foo', d)
print('Should be: {}, is: {}'.format('0',result))
result = get_dictionary_val('~b', d)
print('Should be: {}, is: {}'.format('0',result))
result = get_dictionary_val('~foo', d)
print('Should be: {}, is: {}'.format('1',result))
result = get_dictionary_val('b', d)
print('Should be: {}, is: {}'.format('1',result))

print()
print('Testing expression_eval()')
result = expression_eval(expr3, d)
print('Should be: 0 is {}'.format(result))
result = expression_eval(expr2, d)
print('Should be: 0 is {}'.format(result))
result = expression_eval(expr1, d)
print('Should be: 0 is {}'.format(result))

expr4 = '<=> ~foo b'
result = expression_eval(expr4, d)
print('Should be: 1 is {}'.format(result))

get_dictionary_val() tests:
Should be: 0, is: 0
Should be: 0, is: 0
Should be: 1, is: 1
Should be: 1, is: 1

Testing expression_eval()
Should be: 0 is 0
Should be: 0 is 0
Should be: 0 is 0
Should be: 1 is 1


# Question 3 (Expected) 1 point 


The following code is a recursive evaluator for prefix arithmetic expressions. It assumes that there are always two operands either an integer or a prefix expression starting with an operator (addition or multiplication). It is a good idea to go through this function carefully by hand to understand how the recursion works. 

Informed by your understanding of the arithmetic recursive_eval function your task is to write function to implement a recursive prefix logic evaluator. Your evaluator should also support variables bindings using a dictionary as in the previous question. 

Example expressions: 
```
& 1 & 1 a   
=> 0 & b ~alice  
<=> foo 1 
```

In [89]:
def recursive_eval(l):
    head, tail = l[0], l[1:]
    if head in ['+', '*']: 
        val1, tail = recursive_eval(tail)
        val2, tail = recursive_eval(tail)
        if head == '+': 
            return (int(val1)+int(val2), tail)
        elif head == '*':  
            return (int(val1)*int(val2), tail)
    # operator is a number 
    else:  
        return (int(head),tail)

def prefix_eval(input_str): 
    input_list = input_str.split(' ')
    res, tail = recursive_eval(input_list)
    return res

print(prefix_eval('1'))
print(prefix_eval('+ 1 2'))
print(prefix_eval('+ 1 * 2 3'))
print(prefix_eval('+ * 5 2 * 3 + 1 5'))

1
3
7
28


In [90]:
# YOUR CODE GOES HERE 

def recursive_prefix_logic_eval(expression, dictionary = None):
    #a dictionary has been passed and expression_evaluation will be invoked
    operators = ['&', '|', '=>', '<=>']
    if dictionary != None:
        head, tail = expression[0], expression[1:]
        if head in operators:
            val1, tail = recursive_prefix_logic_eval(tail, dictionary)
            val2, tail = recursive_prefix_logic_eval(tail, dictionary)
            exp = head + ' ' + str(get_dictionary_val(str(val1), dictionary)) + ' ' + str(get_dictionary_val(str(val2), dictionary))
            return (simple_logic_eval(exp), tail)
        else:
            return (get_dictionary_val(head, dictionary), tail)
    
    
    #a dictionary has not been passed which assumes the expression has no variables
    else:
        head, tail = expression[0], expression[1:]
        if head in operators:
            val1, tail = recursive_prefix_logic_eval(tail)
            val2, tail = recursive_prefix_logic_eval(tail)
            exp = head + ' ' + str(val1) + ' ' + str(val2)
            return (simple_logic_eval(exp), tail)
        else:
            return (head, tail)
    
def prefix_logic_eval(expression, dictionary = None):
    expr = expression.split(' ')
    result, tail = recursive_prefix_logic_eval(expr, dictionary)
    return result
    

d = {'alice': 0, 'b':1}    
expr1 = '=> 0 & b ~alice'
expr2 = '<=> 0 | 1 1'
    
result = prefix_logic_eval(expr1, d)
print('Expected: 1, Got: {}'.format(result))
result = prefix_logic_eval(expr2)
print('Expected: 0, Got: {}'.format(result))


Expected: 1, Got: 1
Expected: 0, Got: 0


# QUESTION 4 (EXPECTED) - 1 point


Using the recursive prefix evaluator you defined in the previous question 
answer the following question (you will need to convert the exressions below 
to prefix). You can use multiple string assignments to assemble more complicated 
sentences into one big string: 


Let A be the formula: 

\begin{equation} 
  (( p_{1} \rightarrow (p2 \land p_{3})) \land ((\neg p_{1})
  \rightarrow (p_{3} \land p_{4})))
\end{equation} 

Let B be the formula: 

\begin{equation} 
  (( p_{3} \rightarrow (\neg p_{6})) \land ((\neg
  p_{3}) \rightarrow (p_{4} \rightarrow p_{1})))  
\end{equation} 

Let C be the formula: 

\begin{equation} 
  ((\neg(p2 \land p_{5})) \land (p2 \rightarrow p_{5})) 
\end{equation} 

Let D be the formula: 

\begin{equation} 
  (\neg (p_{3} \rightarrow p_{6}))
\end{equation} 

Evaluate the formulate E: 
\begin{equation} 
  (( A \land (B \land C)) \rightarrow D)
\end{equation} 

under the true assignment $I_{1}$, where $I_{1}(p_{1}) = I_{1}(p_{3}) = I_{1}(p_{5}) = false$ 
and $I_{1}(p2) = I_{1}(p_{4}) = I_{1}(p_{6}) = true$ as well as under the truth assignment 
$I_{2}$, where $I_{2}(p_{1}) = I_{2}(p_{3}) = I_{2}(p_{5}) = true$ and
$I_{2}(p_{2})=I_{2}(p_{4})=I_{2}(p_{6}) = false$. 


In [91]:
# YOUR CODE GOES HERE
truth_assign1 = {'p1': 0, 'p3': 0, 'p5': 0, 'p2': 1, 'p4': 1, 'p6': 1}
truth_assign2 = {'p1': 1, 'p3': 1, 'p5': 1, 'p2': 0, 'p4': 0, 'p6': 0}

A = '& => p1 & p2 p3 => ~p1 & p3 p4'
B = '& => p3 ~p6 => ~p3 => p4 p1'
#used demorgan's law to simplify expression before evaluation
C = '& | ~p2 ~p5 => p2 p5'
#using known logical equivalence (p => q ) <=> ~p v q and demorgans law ~(p v q) <=> ~p & ~q to simplify expression before evaluation
D = '& p3 ~p6'
E = '=> & & => p1 & p2 p3 => ~p1 & p3 p4 & & => p3 ~p6 => ~p3 => p4 p1 & | ~p2 ~p5 => p2 p5 & p3 ~p6'

print('Testing Q4 logical expressions')
print('\nTesting expression A = ' +  A  + ' with truth_assign1 and truth_assign2')
result1 = prefix_logic_eval(A, truth_assign1)
result2 = prefix_logic_eval(A, truth_assign2)
print('Results for A with truth_assign1: {}, expected {}. With truth_assign2: {}, expected: {}'.format(result1, 0, result2, 0))

print('\nTesting expression B = ' +  B  + ' with truth_assign1 and truth_assign2')
result1 = prefix_logic_eval(B, truth_assign1)
result2 = prefix_logic_eval(B, truth_assign2)
print('Results for B with truth_assign1: {}, expected {}. With truth_assign2: {}, expected: {}'.format(result1, 0, result2, 1))

print('\nTesting expression C = ' +  C  + ' with truth_assign1 and truth_assign2')
result1 = prefix_logic_eval(C, truth_assign1)
result2 = prefix_logic_eval(C, truth_assign2)
print('Results for C with truth_assign1: {}, expected {}. With truth_assign2: {}, expected: {}'.format(result1, 0, result2, 1))

print('\nTesting expression D = ' +  D  + ' with truth_assign1 and truth_assign2')
result1 = prefix_logic_eval(D, truth_assign1)
result2 = prefix_logic_eval(D, truth_assign2)
print('Results for D with truth_assign1: {}, expected {}. With truth_assign2: {}, expected: {}'.format(result1, 0, result2, 1))

print('\nTesting expression E = A &(B & C) => D with truth_assign1 and truth_assign2')
result1 = prefix_logic_eval(E, truth_assign1)
result2 = prefix_logic_eval(E, truth_assign2)
print('Results for E with truth_assign1: {}, expected {}. With truth_assign2: {}, expected: {}'.format(result1, 1, result2, 1))



Testing Q4 logical expressions

Testing expression A = & => p1 & p2 p3 => ~p1 & p3 p4 with truth_assign1 and truth_assign2
Results for A with truth_assign1: 0, expected 0. With truth_assign2: 0, expected: 0

Testing expression B = & => p3 ~p6 => ~p3 => p4 p1 with truth_assign1 and truth_assign2
Results for B with truth_assign1: 0, expected 0. With truth_assign2: 1, expected: 1

Testing expression C = & | ~p2 ~p5 => p2 p5 with truth_assign1 and truth_assign2
Results for C with truth_assign1: 0, expected 0. With truth_assign2: 1, expected: 1

Testing expression D = & p3 ~p6 with truth_assign1 and truth_assign2
Results for D with truth_assign1: 0, expected 0. With truth_assign2: 1, expected: 1

Testing expression E = A &(B & C) => D with truth_assign1 and truth_assign2
Results for E with truth_assign1: 1, expected 1. With truth_assign2: 1, expected: 1


# QUESTION 5 (ADVANCED) - 1 point 

Implement inference using model-checking using your prefix recursive evaluator to decide whether a knowledge base KB entais some sentence a. To do so express the knowledge base in the prefix notation, enumerate all models for the variables in the dictionary, and check that the sentence a is true in every model in which the KB is true. 

You can check the implementation to tt_entails in logic.ipynb in the aima_python repository to inform how you implement your solution. Your solution should NOT rely directly on any code in logic.py or logic.ipynb. 

Check your model checking using the examples that are used in logic.ipynb to check entailment (there are a few with P and Q as variables as well as the one with A, B, C, D, E, F, G. You will need to convert these examples to prefix notation. 


In [92]:
# YOUR CODE GOES HERE 
from itertools import product


#assumes a list of uniqe variables and generates a list of dictionaries with all possible truth assignments to each variable
def dict_generator(lov):
    lod = []
    n = len(lov)
    for i in product([0,1], repeat=n):
        new_dict = {} 
        for j in i:     
            for k in lov:
                new_dict[k] = j
        lod.append(new_dict)
    return lod


#returns a list of the models in which the sentence wiht the given is entailed
def entail_check(lod, sentence_a, given):
    true_models =[]
    result1 = []
    
    for d in lod:
        if d[given] == 1:
            true_models.append(d)
        else:
            pass
    for i in true_models:
        result = prefix_logic_eval(sentence_a, i)
        if result:
            result1.append(i)
        else:
            pass
    return result1


A = '& P Q'
lov3 = ['P', 'Q']
given = 'Q'
result = entail_check(dict_generator(lov3), A, given)
print('Sentence {} given: {}, is entailed in models which have the following assinments: {}'.format(A, given, result))

A = '| P Q'
lov3 = ['P', 'Q']
given = 'Q'
result = entail_check(dict_generator(lov3), A, given)
print('Sentence {} given: {} is entailed in models which have the following assinments: {}'.format(A, given, result))

A = '| Q P'
lov3 =['P', 'Q']
given = 'P'
result = entail_check(dict_generator(lov3), A, given)
print('Sentence {} given: {} is entailed in models which have the following assinments: {}'.format(A, given, result))



Sentence & P Q given: Q, is entailed in models which have the following assinments: [{'P': 1, 'Q': 1}, {'P': 1, 'Q': 1}]
Sentence | P Q given: Q is entailed in models which have the following assinments: [{'P': 1, 'Q': 1}, {'P': 1, 'Q': 1}]
Sentence | Q P given: P is entailed in models which have the following assinments: [{'P': 1, 'Q': 1}, {'P': 1, 'Q': 1}]


# Extra ideas (no credit) 

* Implement conversion of the prefix expressions to prefix conjuctive normal form (CNF) based on the recursive evaluator you have implemented. 
* Based on the recursive evaluator you have implemented do a conversion of expressions in prefix notation to the infix notation of expressions supported by logic.ipynb. Provide 4 test cases that demonstrate the the conversion works by confirming that the result of your evaluator and the logic.ipynb evaluator are the same. 



# Question 6 (Basic) -1 point

Consider the following propositional logic knowledge base.

It is not sunny this afternoon and it is colder than yesterday.
We will go swimming only if it is sunny.
If we do not go swimming then we will take a canoe trip.
If we take a canoe trip, then we will be home by sunset.
Denote:

* p = It is sunny this afternoon
* q = it is colder than yesterday
* r = We will go swimming
* s= we will take a canoe trip
* t= We will be home by sunset

Express this knowledge base using propositional logic using the expression syntax used in logic.ipynb. You can incoprorate any code you need from logic.ipynb and logic.py. Using both model checking and theorem proving inference (you can use the implementations providedin logic.py) show that this knowledge base entails the sentence if it is not sunny this afternoon then we will be home by sunset. 

In [93]:
# YOUR CODE GOES HERE 
%cd ../aima-python
from utils import * 
from logic import *
import agents as a

prop_KB = PropKB()

P, Q, R, S, T = symbols('P, Q, R, S, T')
prop_KB.tell(~P | '&' | Q)
prop_KB.tell(R | '<=>' | P)
prop_KB.tell(~R | '==>' | S)
prop_KB.tell(S | '==>' | T)

print('Model Checking with tt_entails:')
print(tt_entails(~P & Q & (R | ~P) & (P | ~R) & (S | R) & (T | S) , expr(~P | T)))
print('Theorem Proving with pl_resolution:')
print(pl_resolution(prop_KB, (~P | '==>' | T)))


/home/jovyan/aima-python
Model Checking with tt_entails:
True
Theorem Proving with pl_resolution:
True


# Question 7 (Basic)  - 1 point 

Encode the kindship domain described in section 8.3.2 of the textbook using FOL and FolKB implementation in logic.py and encode as facts the relationships between the members of the Simpsons family from the popular TV show:  

https://en.wikipedia.org/wiki/Simpson_family


Show how the following queries can be answered using this KB: 

* Who are the children of Homer ? 
* Who are the parents of Bart ? 
* Are Lisa and Homer siblings ? 
* Are Lisa and Bart siblings ? 


In [94]:
# YOUR CODE GOES HERE 
%cd ../aima-python
from utils import * 
from logic import *
import agents as a

clauses = []

clauses.append(expr('Parent(p,c) ==> Child(c, p)'))
clauses.append(expr('Child(c, p) ==> Parent(p,c)'))
clauses.append(expr('Spouse(h, w)'))
clauses.append(expr("Siblings(y, x) ==> Siblings(x, y)"))
clauses.append(expr("Siblings(x, y) ==> Siblings(y, x)"))
clauses.append(expr('Parent(p, c) & Spouse(p, w) ==> Parent(w, c)'))
clauses.append(expr('Parent(p, c) & Parent(p, k) & Siblings(c, k) ==> Siblings(c, k)'))

clauses.append(expr('Parent(Homer, Bart)'))
clauses.append(expr('Parent(Homer, Hugo)'))
clauses.append(expr('Parent(Homer, Maggie)'))
clauses.append(expr('Parent(Homer, Lisa)'))
clauses.append(expr('Siblings(Bart, Lisa)'))
clauses.append(expr('Spouse(Homer, Marge)'))



Simpsons = FolKB(clauses)




children = fol_fc_ask(Simpsons, expr('Child(x, Homer)'))
print('Homer\'s chilren are: ', list(children))
b_parents = fol_fc_ask(Simpsons, expr('Parent(p, Bart)'))
print('Bart\'s parents are: ', list(b_parents))
siblings = fol_fc_ask(Simpsons, expr('Siblings(Homer, Lisa)'))
print('Are Lisa and Homer siblings? ', bool(list(siblings)))
siblings = fol_fc_ask(Simpsons, expr('Siblings(Bart, Lisa)'))
print('Are Lisa and Bart Siblings? ', bool(list(siblings)))




/home/jovyan/aima-python
Homer's chilren are:  [{x: Hugo}, {x: Maggie}, {x: Bart}, {x: Lisa}]
Bart's parents are:  [{p: Homer}, {p: Marge}]
Are Lisa and Homer siblings?  False
Are Lisa and Bart Siblings?  True


# QUESTION 8 (EXPECTED) 1 point

In this question we explore Prolog which is a programming language based on logic. We won't go into details but just wanted to give you a flavor of the syntax and how it connects to what we have learned. For this question you 
will NOT be using the notebook so your answer should just be the source code. We will use http://tau-prolog.org/ which is a Prolog implementation that can run in a browser. When you access the webpage there is a text window labeled try it for entering your knowledge base and under it t
here is a text entry field for entering your query. 

For example type in the Try it window and press enter: 

```Prolog
likes(sam, salad).
likes(dean, pie).
likes(sam, apples).
likes(dean, whiskey).
```

Then enter the query: 
```Prolog 
likes(sam,X).
```
When you press Enter once you will get X=apples. and X=salad. Note the periods at the end of each statement. 

Encode the kinship domain from the previous question in Prolog and answer the queries from the previous question. Notice that in Prolog the constants start with lower case letters and the variables start with upper case letters.

Provide your code for the KB and queries using markup. See the syntax for Prolog of this cell by double clicking for editing. 



# YOUR CODE GOES HERE (USING MARKDOWN)

parent(P, C) :- child(C,P).
child(C, P) :- parent(P, C).
spouse(H, W). 
siblings(Y, X) :- siblings(X, Y).
siblings(X, Y) :- siblings(Y, X). 
parent(P, C), spouse(P, W) :- parent(W, C).
parent(P, C), parent(P, K), siblings(C, K) :- siblings(K, C).

parent(homer, bart).
parent(homer, lisa).
parent(homer, maggie).
parent(homer, hugo).
siblings(bart, lisa).
spouse(homer, marge).


#Queries:

#Have to hit enter 4 times to get all answers to query 1 on tau-prolog
child(homer, X).
parent(bart, P).
?- siblings(homer, lisa). 
?- siblings(bart, lisa).


# QUESTION 9 (EXPECTED) 1 point 


## Legoworld 


In this question we explore the use of FOL to encode knowledge about the objects 
and arrangement of a simple world created by different lego pieces. Our world 
will consist of making simple structure by placing lego bricks on top of each other. 
Each lego brick will be identified by a unique identifier (the letters in the figure below). 

Let's look at a specific example where each brick is labeled by a letter: 

<img src="lego_letters.png" width="60%"/>

This corresponds to the following picture: 

<img src="lego2.png" width="60%"/>




We can use the following predicates to model the world: 
* OnPlate(b): b is attached to the bottom plate 
* On(b1,b2): brick b1 is placed on top of brick b2 
* AtLeft(b1,b2): bricks b1 and b2 are placed on the top, and block b1 is direct at the left of b2 
* Color(b,c): The color of brick b is c 
* Cype(b,t): The type of brick b is c (Brick, Plate, Tile) 

<img src="lego3.png" width="20%"/>


Each brick will be identified by the names appearing in the picture. The thicker brick with studs will be called Brick, the thinner brick with studs is a plate, and the one that is flat is a tile. 



The task to do using the FOL KB implementation in logic.py is:

1. Write a database of facts which models the world in the picture. For example you can use clauses.append(expr('TypeOf(A,Brick)')) to state that A is a Brick. 

2. Based exclusively on using these predicates, definte the following predicates:
    * Base(b1, b2): b2 is the base of the tower containing b1.
    * Base_at_right(b1, b2): b1 and b2 are on the plate, and b2 is at the right (but perhaps not directly) of b1.
    * Object_at_right(b1, b2): b1 is in a pile which is at the right (but perhaps not directly) of b2.
    
    
The above predicates must work for any world defined using the facts specified by on_plate, on, 
at_left not just the specific example encoded above. In other words these predicates should be defined 
in terms of the existing predicates and variables. As an example here is the definition of Base. 

    * clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
    * clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))
    
This is a recursive definition it is a good idea to see how it works by doing substitutions by hand. 


3. Using the KB you created answer the following queries: 
    * Is brick B on top of brick C 
    * What is the type and color of the brick on top of C 
    * What is the type of the base of H
    * What are the objects that are right of C






In [ ]:
#YOUR CODE GOES HERE
%cd ../aima-python
from utils import * 
from logic import *
import agents as a


clauses = []

#types and colors of bricks
clauses.append(expr('Type(A, Brick)'))
clauses.append(expr('Color(A, Red)'))
clauses.append(expr('Type(B, Tile)'))
clauses.append(expr('Color(B, White)'))
clauses.append(expr('Type(C, Plate)'))
clauses.append(expr('Color(C, Brown)'))
clauses.append(expr('Type(D, Brick)'))
clauses.append(expr('Color(D, Grey)'))
clauses.append(expr('Type(E, Tile)'))
clauses.append(expr('Color(E, Brown)'))
clauses.append(expr('Type(F, Brick)'))
clauses.append(expr('Color(F, Red)'))
clauses.append(expr('Type(G, Tile)'))
clauses.append(expr('Color(G, Brown)'))
clauses.append(expr('Type(H, Brick)'))
clauses.append(expr('Color(H, Red)'))
clauses.append(expr('Type(I, Brick)'))
clauses.append(expr('Color(I, Blue)'))
clauses.append(expr('Type(J, Plate)'))
clauses.append(expr('Color(J, Yellow)'))

#positions of legos
clauses.append(expr('OnPlate(A)'))
clauses.append(expr('OnPlate(D)'))
clauses.append(expr('OnPlate(F)'))
clauses.append(expr('OnPlate(J)'))

clauses.append(expr('On(C, D)'))
clauses.append(expr('On(B, C)'))
clauses.append(expr('On(E, F)'))
clauses.append(expr('On(I, J)'))
clauses.append(expr('On(H, I)'))
clauses.append(expr('On(G, H)'))

clauses.append(expr('AtLeft(F, J)'))
clauses.append(expr('AtLeft(D, F)'))
clauses.append(expr('AtLeft(A, D)'))



#definition of new predicates
#base
clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))

#base_at_right
clauses.append(expr('AtLeft(b1, b2) ==> Base_at_right(b1, b2)')) #b2 is directly to the right of b1
clauses.append(expr('Base_at_right(b, b1) & AtLeft(b1, b2) ==> Base_at_Right(b, b2)')) #b1 is to the right of b and to the left of b2 so b2 is right of b1
clauses.append(expr('Base_at_Right(A, D)'))


#object_at_right
clauses.append(expr('On(y,x) & On(z, y) ==> On(z,x)')) #y is on x and z is on y so z is on x
clauses.append(expr('Base_at_right(b1, b2) & On(p1, b2) ==> Object_at_right(b1, b2)')) #b2 is the object to the right of b1
clauses.append(expr('Base(b1) ==> On(p1, b1)'))

LegoWorld = FolKB(clauses)

print('LegoWorld Questions:')

result = pl_resolution(LegoWorld, expr('On(B,C)'))
print('Is brick B on brick C? {}'.format(result))

result1 = fol_fc_ask(LegoWorld, expr('On(x, C)'))
result2 = fol_fc_ask(LegoWorld, expr('Color(B, x)'))
result3 = fol_fc_ask(LegoWorld, expr('Type(B, x)'))
print('The brick on top of brick C is: {}. The color of the brick on brick C is: {}, and the type is: {}'.format(list(result1), list(result2), list(result3)))

result1 = fol_fc_ask(LegoWorld, expr('Base(H, x)'))
result2 = fol_fc_ask(LegoWorld, subst({x: list(result1)[0][x]}, expr('Type(x, y)')))
print('The type of brick at the base of brick H is: {}'.format(list(result2)))

#result = fol_fc_ask(LegoWorld, expr('Base(C, x)'))
#result2 = fol_fc_ask(LegoWorld, subst({x: list(result)[0][x]}, expr('Base_at_right(x, z)'))) #base to right of c
#result3 = fol_fc_ask(LegoWorld, subst({z: list(result2)[0][z]}, expr('On(x, z)'))) #on f
#print('The objects to the right of Brick C are: {}{}'.format(list(result2),list(result3)))


/home/jovyan/aima-python
LegoWorld Questions:
Is brick B on brick C? True


# QUESTION 10 (ADVANCED) 1 point 


This question is more advanced and open ended. I provide two options. 

## Option 1 

Extend the Legoworld knowledge base with a predicate to determine if a brick is part of an unstable 
tower. Any brick placed on top of a tile results in an unstable tower. For example the brown plate 
and grey brick in the middle are unstable but the red brick under the tile in the middle is stable. 
This is a trickier predicate to define so show a few cases to ensure that it works as expected. 

<img src="lego1.png" width="40%"/>

Provide a simple natural language processing interface to the LegoWorld that takes input from 
the user and returns the results in more natural lanuage. For example you could have the following 
dialog: 

* User: What color is brick B ? 
* Computer: Brick B is Red. 
* User: Is there a brick that is on top of a tile? 
* Computer: Yes, brick D is on top of a tile. 
etc 

You will basically write a simple translation layer from simplified English to FOL tell and ask requests and back to simplified English. 


This question is inspired by the classic natural understanding work using logic: https://en.wikipedia.org/wiki/SHRDLU



## Option 2 


This question explores the automatic constructions of a first-order logic knowledge base from a web resource and is more open ended than the other ones. The website https://www.songfacts.com/ contains a large variety of facts about music. Check the https://www.songfacts.com/categories link for some categories. Using selenium Python bindings https://selenium-python.readthedocs.io/ access the webpage and scrape at least three categories. Your code should scrape the information from the pages and convert it into relationships and facts in first-order logic using the syntax of expressions in logic.ipynb. Once you build your knowledge-base then write 4 non-trivial queries that show-case the expressiveness of FOL. These queries should not be possible to be answered easily using the web interface i.e they should have some logical connectives, more than one possible answer etc. The translation of the song facts from the web page to FOL should NOT be done by hand but using the web scraping tool you develop. You can use multiple cells in your answer.